# starGAN 논문 구현

In [2]:
import tensorflow as tf

In [3]:
from sklearn.preprocessing import OneHotEncoder

# Load dataset

In [22]:
def random_image_label_data(sample_size, image_size, RGB=3):
    '''
    sample_size : 원하는 샘플 양 설정.
    image_size :  원하는 이미지 사이즈 설정. ex) 128x128 -> image_size=128
    RGB : default=3 
    '''
    N1 = sample_size
    image_size_ = image_size
    RGB_ = RGB
    # np.random.normal : Draw random samples from a normal (Gaussian) distribution.
    image_data = np.random.normal(size=[N1,image_size_,image_size_,RGB_]) 
    label_hair = tf.one_hot(np.random.randint(0,3,size=N1),depth=3)
    label_gender = tf.one_hot(np.random.randint(0,2,size=N1),depth=1)
    label_age = tf.one_hot(np.random.randint(0,2,size=N1),depth=1)
    label_gender_age = np.append(label_gender,label_age, axis=1)
    label = np.append(label_hair,label_gender_age, axis=1)
    return (image_data, label)

In [23]:
# 데이터 shape 파악
# input이 들어가서 output으로 오류없이 나오는지만 확인._실제로 하면 GPU필요하고 하루 이상 돌아가야함.
image_data, label = random_image_label_data(1000,128)
train_dataset = tf.data.Dataset.from_tensor_slices((image_data, label))
train_dataset = train_dataset.batch(2) # 작은 배치로 해서 코드가 잘 돌아가는지 점검

for images, labels in train_dataset.take(2): 
    print(images.shape)
    print(labels.shape)
    '''
    (2, 128, 128, 3) #첫번째 2개씩 128,128,3짜리
    (2, 5) # 첫번째 label 2개씩 불러오기
    (2, 128, 128, 3)
    (2, 5)
    '''

(2, 128, 128, 3)
(2, 5)
(2, 128, 128, 3)
(2, 5)


# Create the Models

In [ ]:
# class 만들때, tf.keras.model ; 반드시 상속
# Appendix에 나온 대로 하면됨

## Generator

### Down sampling

- concat 함수

In [24]:
def concat(image, label):
    '''
    1차원 행렬인 label(array)을 다차원 행렬인 image(array)에 맞춰서 broadcasting해서 합쳐서 tensor로 출력해주는 함수.
    label, image 는 array와 tensor형태 모두 허용.
    '''
    N = len(image) #  총 샘플 수
    label_len =  len(label[0]) # 하나의 label의 list 길이 
    image_size = len(image[0]) # 하나의 image의 한변의 길이

    #비어있는 list 정의
    concat = []
    for i in range(N):
        # label과 image i번째 성분
        label_ = label[i]
        image_ = image[i]
        # 128x128x5짜리 성분이 1인 행렬 만들기
        ones = np.ones((image_size, image_size, label_len))
        # ones에서 [1,1,1,1,1]을 label_로 치환해서 128x128x5짜리 label 행렬 구축.
        label_vol = np.where(ones==np.ones((label_len)), label_ , ones)
        # image데이터와 label데이터를 concat해서 128x128x8행렬로 변경
        image_label_concat = np.concatenate((image_, label_vol), axis=2)
        # image_label_concat을 concat에 추가.
        concat.append(image_label_concat)
    # concat을 list에서 np.array로 바꾸고 출력.
    concat_array = np.array(concat) 
    # list를 바로 텐서로 바꾸니 시간이 오래걸려서 array 바꾸고 tensor로 바꾸는 방식으로.
    return tf.convert_to_tensor(concat_array, dtype=tf.float32) # tensor형태로 바꿔서 출력

In [25]:
xxx = concat(image_data, label)
#xxx

In [ ]:
class Downsampling(tf.keras.Model):
    def __int__(self):
        super(Downsampling, self).__init__()
        self.conv1 = Conv2D(64, kernel_size = 7, strides = 1, padding = 'SAME')
        self.conv2 = Conv2D(128, kernel_size = 4, strides = 2, padding = 'SAME')
        self.conv3 = Conv2D(256, kernel_size = 4, strides = 2, padding = 'SAME')
        self.IN = InstanceNormalization() #구현필요
        self.activation = layers.relu()
    
    def call(self, image, label):
        x = concat(image,label) #구현필요
        x = self.conv1(x)
        x = self.IN(x)
        x = self.activation(x)
        x = self.conv2(x)
        x = self.IN(x)
        x = self.activation(x)
        x = self.conv3(x)
        x = self.IN(x)
        x = self.activation(x)
        return x

# loss

In [16]:
'''
cf) tf2.0에서 텐서 연산 예시
'''
tf.executing_eagerly()
test = [[2.,-1.,4.]]
sol = [[1.,4.,1.]]
tf.norm(tf.math.subtract(test,sol),axis=0)

<tf.Tensor: id=52, shape=(3,), dtype=float32, numpy=array([1., 5., 3.], dtype=float32)>

In [15]:
tf.math.subtract(test,sol)

<tf.Tensor: id=43, shape=(1, 3), dtype=float32, numpy=array([[ 1., -5.,  3.]], dtype=float32)>

In [20]:
'''
loss 함수, D(c'|x) 이런거 어떻게 구현하지?
'''

generated_output = generator(concat(real_image, label_fake)) #: fake image

def adv_loss(real_output, generated_output):
    D_real = discriminator(real_output)  # real_output = real_image + cls 정답
    D_fake = discriminator(generated_output) # generated_input = generated_image + cls 정답
    return tf.reduce_mean(tf.log(D_real)+tf.log(1-D_fake))
    
def real_cls_loss(real_input): #?
    D_real = discriminator(real_input) # real_input = real_image + cls 정답
    return tf.reduce_mean(-tf.log(D_real))

def fake_cls_loss(generated_output, label_fake): #cls 들어가야하는거 맞나?
    D_fake = discriminator(label_fake, generated_output) # real_input = real_image + cls 정답
    return tf.reduce_mean(-tf.log(D_fake))

def rec_loss(real_output, generated_output, label_real):
    G_real_from_fake = generator(conct(generated_output, label_real))
    real_subt_G = tf.norm(tf.math.subtract(real_output,G_real_from_fake), axis=1 )#axis 수정??? 
    return tf.reduce_mean(real_subt_Rfake)

def generator_loss(generated_output, lambda_cls, lambda_rec):
    return adv_loss(real_output, generated_output) + lambda_cls*fake_cls_loss(generated_output, label_fake)\
        +lambda_rec*rec_loss(real_output, generated_output, label_real)

def discriminator_loss(real_output, generated_output,  lambda_cls, real_input):
    return -adv_loss(real_output, generated_output) + lambda_cls*real_cls_loss(real_input)



NameError: name 'generator' is not defined

In [109]:
concat = []
image_size = 128
label_len = 5
i=0

label_ = label.numpy()[i]
image_ = image_data[i]
ones = np.ones((image_size, image_size, label_len))
label_vol = np.where(ones==np.ones((1,label_len)), label_ , ones)
image_label_concat = np.concatenate((image_, label_vol), axis=2)
concat.append(image_label_concat)
np.array(concat).shape

(1, 128, 128, 8)

In [23]:
for images, labels in train_dataset.take(1): 
    output = conv(images)
    print(output.shape)
#     print(labels.shape)

(2, 64, 64, 4)


In [ ]:
class MyModel(tf.keras.Model):

    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        # Define your layers here.
        self.dense_1 = layers.Dense(32, activation='relu')
        self.dense_2 = layers.Dense(num_classes, activation='sigmoid')

    def call(self, inputs):
        # Define your forward pass here,
        # using layers you previously defined (in `__init__`).
        x = self.dense_1(inputs)
        return self.dense_2(x)



## Discriminator

# Define the loss functions and the optimizer

In [6]:
# 논문에서 G와 D 각각 loss를 어떻게 구현하는지 파악해야함